In [1]:
import argparse
import os

import apache_beam as beam
import tensorflow as tf
from apache_beam.options.pipeline_options import PipelineOptions
import apache_beam.runners.interactive.interactive_beam as ib
import apache_beam.transforms.sql

import beam__common
import fidscs_globals

import data_extractor
import preprocessor

# from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow.keras.preprocessing.image

In [2]:
data_dir = "/tmp/fids-capstone-data/data"

In [3]:
data_extractor.run(max_target_videos=-1, data_dir=data_dir, use_beam=True)

use_beam: True
Found dataset /tmp/fids-capstone-data/data/consultant-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-targetvideo-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-utterance-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-utterance-targetvideo-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-utterance-token-index.csv
Found dataset /tmp/fids-capstone-data/data/ncslgr-corpus-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-targetvideo-utterance-token-frame-index.csv
Found dataset /tmp/fids-capstone-data/data/vocabulary-index.csv


In [4]:
preprocessor.run(data_dir=data_dir)

Found train/val dataset /tmp/fids-capstone-data/data/train-assoc.csv
Found train/val dataset /tmp/fids-capstone-data/data/val.csv
Found train/val dataset /tmp/fids-capstone-data/data/train.csv


In [5]:
options = {
    'project': 'my-project', # change
    'runner': 'InteractiveRunner',
    'direct_num_workers': 0, # 0 is use all available cores
    'direct_running_mode': 'multi_threading', # ['in_memory', 'multi_threading', 'multi_processing'] # 'multi_processing' doesn't seem to work for DirectRunner?
    'streaming': False # set to True if data source is unbounded (e.g. GCP PubSub)
}
pipeline_options = PipelineOptions(flags=[], **options) # easier to pass in options from command-line this way
print(f"PipelineOptions:\n{pipeline_options.get_all_options()}\n")

PipelineOptions:
{'runner': 'InteractiveRunner', 'streaming': False, 'beam_services': {}, 'type_check_strictness': 'DEFAULT_TO_ANY', 'type_check_additional': '', 'pipeline_type_check': True, 'runtime_type_check': False, 'performance_runtime_type_check': False, 'direct_runner_use_stacked_bundle': True, 'direct_runner_bundle_repeat': 0, 'direct_num_workers': 0, 'direct_running_mode': 'multi_threading', 'dataflow_endpoint': 'https://dataflow.googleapis.com', 'project': 'my-project', 'job_name': None, 'staging_location': None, 'temp_location': None, 'region': None, 'service_account_email': None, 'no_auth': False, 'template_location': None, 'labels': None, 'update': False, 'transform_name_mapping': None, 'enable_streaming_engine': False, 'dataflow_kms_key': None, 'flexrs_goal': None, 'hdfs_host': None, 'hdfs_port': None, 'hdfs_user': None, 'hdfs_full_urls': False, 'num_workers': None, 'max_num_workers': None, 'autoscaling_algorithm': None, 'machine_type': None, 'disk_size_gb': None, 'disk_t

In [6]:
fidscs_globals.DATA_ROOT_DIR = data_dir

In [7]:
can_proceed = True

if not tf.io.gfile.exists(fidscs_globals.DATA_ROOT_DIR) or len(tf.io.gfile.listdir(fidscs_globals.DATA_ROOT_DIR))==0:
    print(f"{fidscs_globals.VALIDATION_FATAL_ERROR_TEXT} data directory does not exist or is empty!")
    can_proceed = False
else:
    fidscs_globals.VIDEO_DIR = os.path.join(fidscs_globals.DATA_ROOT_DIR, 'videos')
    fidscs_globals.STICHED_VIDEO_FRAMES_DIR = os.path.join(fidscs_globals.DATA_ROOT_DIR, 'stitched_video_frames')
    fidscs_globals.CORPUS_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.CORPUS_DS_FNAME)
    fidscs_globals.DOCUMENT_ASL_CONSULTANT_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.DOCUMENT_ASL_CONSULTANT_DS_FNAME)
    fidscs_globals.ASL_CONSULTANT_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.ASL_CONSULTANT_DS_FNAME)
    fidscs_globals.VIDEO_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VIDEO_DS_FNAME)
    fidscs_globals.VIDEO_SEGMENT_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VIDEO_SEGMENT_DS_FNAME)
    fidscs_globals.VIDEO_FRAME_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VIDEO_FRAME_DS_FNAME)
    fidscs_globals.UTTERANCE_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_DS_FNAME)
    fidscs_globals.UTTERANCE_VIDEO_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_VIDEO_DS_FNAME)
    fidscs_globals.UTTERANCE_TOKEN_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_TOKEN_DS_FNAME)
    fidscs_globals.UTTERANCE_TOKEN_FRAME_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_TOKEN_FRAME_DS_FNAME)
    fidscs_globals.VOCABULARY_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VOCABULARY_DS_FNAME)
    fidscs_globals.TRAIN_ASSOC_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.TRAIN_FRAME_SEQ_ASSOC_DS_FNAME)
    fidscs_globals.VAL_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VAL_FRAME_SEQ_DS_FNAME)
    fidscs_globals.TRAIN_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.TRAIN_FRAME_SEQ_DS_FNAME)

In [8]:
pl = beam.Pipeline(options=pipeline_options)

def train_val_index_schemad_pcoll_rows__to__ordered_tuples(train_val_index_schemad_pcoll_row):
    """
    """
    return (
        # SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX = [
        #     'TokenID',
        #     'CameraPerspective',
        #     'ASLConsultantID',
        #     'TargetVideoFilename',
        #     'UtteranceSequence',
        #     'TokenSequence',
        #     'FrameSequence'
        # ]
        train_val_index_schemad_pcoll_row.TokenID,
        train_val_index_schemad_pcoll_row.CameraPerspective,
        train_val_index_schemad_pcoll_row.ASLConsultantID,
        train_val_index_schemad_pcoll_row.TargetVideoFilename,
        train_val_index_schemad_pcoll_row.UtteranceSequence,
        train_val_index_schemad_pcoll_row.TokenSequence,
        train_val_index_schemad_pcoll_row.FrameSequence
    )

train_frame_sequences__assoc_index_schemad_pcoll = beam__common.pl__1__read_train_frame_sequences__assoc_index_csv(pl)
train_frame_sequences__assoc_index = (
    train_frame_sequences__assoc_index_schemad_pcoll
    | "Beam PL: transform train_frame_sequences__assoc_index_schemad_pcoll rows to ordered tuples (according to schema)" >> beam.Map(train_val_index_schemad_pcoll_rows__to__ordered_tuples)
)

val_frame_sequences_index_schemad_pcoll = beam__common.pl__1__read_val_frame_sequences__index_csv(pl)
val_frame_sequences_index = (
    val_frame_sequences_index_schemad_pcoll
    | "Beam PL: transform val_frame_sequences_index_schemad_pcoll rows to ordered tuples (according to schema)" >> beam.Map(train_val_index_schemad_pcoll_rows__to__ordered_tuples)
)

train_frame_sequences_index_schemad_pcoll = beam__common.pl__1__read_train_frame_sequences_index_csv(pl)
train_frame_sequences_index = (
    train_frame_sequences_index_schemad_pcoll
    | "Beam PL: transform train_frame_sequences_index_schemad_pcoll rows to ordered tuples (according to schema)" >> beam.Map(train_val_index_schemad_pcoll_rows__to__ordered_tuples)
)

In [9]:
def train_val_index__load_frame_img(train_val_index_tpl):
    """
    SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX = [
        'TokenID',
        'CameraPerspective',
        'ASLConsultantID',
        'TargetVideoFilename',
        'UtteranceSequence',
        'TokenSequence',
        'FrameSequence'
    ]
    """
    target_video_frames_dir = os.path.join(
        fidscs_globals.STICHED_VIDEO_FRAMES_DIR, 
        train_val_index_tpl[3].split('.')[0]
    )
    target_video_frame_fname = f"{train_val_index_tpl[6]}.jpg"
    target_video_frame_path = os.path.join(target_video_frames_dir, target_video_frame_fname)
    target_video_frame_img = tf.keras.preprocessing.image.load_img(
        target_video_frame_path, 
        target_size=fidscs_globals.FRAME_IMG_INPUT_SHAPE
    )  # this is a PIL image
    target_video_frame_img_array = tf.keras.preprocessing.image.img_to_array(target_video_frame_img)
    target_video_frame_img_array = target_video_frame_img_array.reshape((1,) + target_video_frame_img_array.shape)
    target_video_frame_img_array /= 255.0 # Rescale by 1/255

    return (
        train_val_index_tpl[0],
        train_val_index_tpl[1],
        train_val_index_tpl[2],
        train_val_index_tpl[3],
        train_val_index_tpl[4],
        train_val_index_tpl[5],
        train_val_index_tpl[6],
        target_video_frame_img_array
    )

In [10]:
def pl__X__sample_train_val_frame_sequences_index__with_frame_tensors(train_val_frame_sequences_index, name_train_val_frame_sequences_index, n_sample_size=50):
    """
    """
    return (
        train_val_frame_sequences_index
        | f"Beam PL: get random sample of {n_sample_size} frames from {name_train_val_frame_sequences_index}" >> beam.combiners.Sample.FixedSizeGlobally(n_sample_size)
        | f"Beam PL: flatten random sample of {n_sample_size} frames from {name_train_val_frame_sequences_index}" >> beam.FlatMap(lambda lst_tpl: lst_tpl)
        | f"Beam PL: load frame images of {name_train_val_frame_sequences_index}" >> beam.Map(train_val_index__load_frame_img)
    )

train_frame_sequences__assoc_index__sample__with_frame_tensors = pl__X__sample_train_val_frame_sequences_index__with_frame_tensors(
    train_frame_sequences__assoc_index, 
    "train_frame_sequences__assoc_index"
)
val_frame_sequences_index__sample__with_frame_tensors = pl__X__sample_train_val_frame_sequences_index__with_frame_tensors(
    val_frame_sequences_index, 
    "val_frame_sequences_index"
)
train_frame_sequences_index__sample__with_frame_tensors = pl__X__sample_train_val_frame_sequences_index__with_frame_tensors(
    train_frame_sequences_index, 
    "train_frame_sequences_index"
)

In [11]:
# we require this in order to make use of ib.show() (which provides visualization of the pcolls specified) or ib.collect() (which creates a pandas dataframe from a pcoll)
    # but all pcolls we wish to visualize must be created prior to executing the following line
ib.watch(locals())

<p><br>

##### Show base train/val frame datasets

In [12]:
df_train_frame_sequences__assoc_index = ib.collect(train_frame_sequences__assoc_index)

In [13]:
df_train_frame_sequences__assoc_index.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX
df_train_frame_sequences__assoc_index.set_index(fidscs_globals.SCHEMA_PK__TRAIN_OR_VAL_INDEX, inplace=True)
df_train_frame_sequences__assoc_index.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_train_frame_sequences__assoc_index.sort_index(inplace=True)

df_train_frame_sequences__assoc_index

FrameSequence
TokenID CameraPerspective ASLConsultantID TargetVideoFilename        UtteranceSequence TokenSequence               
0       0                 2               roadtrip2_1051_small_0.mov 44                0                       5348
                                                                                       0                       5349
                                                                                       0                       5350
                                                                                       0                       5351
                                                                                       0                       5352
...                                                                                                             ...
2397    2                 2               football_1073_small_2.mov  31                5                       3525
                                                                                       5                       3526
                                                                                       5                       3527
                                                                                       5                       3528
                                                                                       5                       3529

[196053 rows x 1 columns]

In [14]:
df_val_frame_sequences_index = ib.collect(val_frame_sequences_index)

In [15]:
df_val_frame_sequences_index.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX
df_val_frame_sequences_index.set_index(fidscs_globals.SCHEMA_PK__TRAIN_OR_VAL_INDEX, inplace=True)
df_val_frame_sequences_index.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_val_frame_sequences_index.sort_index(inplace=True)

df_val_frame_sequences_index

FrameSequence
TokenID CameraPerspective ASLConsultantID TargetVideoFilename         UtteranceSequence TokenSequence               
0       0                 6               ben_story_441_small_0.mov   14                2                       1090
                                                                                        2                       1091
                                                                                        2                       1092
                                                                                        2                       1093
                                                                                        2                       1094
...                                                                                                              ...
2397    2                 2               lapd_story_1083_small_2.mov 86                8                       6519
                                                                                        8                       6520
                                                                                        8                       6521
                                                                                        8                       6522
                                                                                        8                       6523

[41181 rows x 1 columns]

In [16]:
df_train_frame_sequences_index = ib.collect(train_frame_sequences_index)

In [17]:
df_train_frame_sequences_index.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX
df_train_frame_sequences_index.set_index(fidscs_globals.SCHEMA_PK__TRAIN_OR_VAL_INDEX, inplace=True)
df_train_frame_sequences_index.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_train_frame_sequences_index.sort_index(inplace=True)

df_train_frame_sequences_index

FrameSequence
TokenID CameraPerspective ASLConsultantID TargetVideoFilename        UtteranceSequence TokenSequence               
0       0                 2               roadtrip2_1051_small_0.mov 44                0                       5348
                                                                                       0                       5349
                                                                                       0                       5350
                                                                                       0                       5351
                                                                                       0                       5352
...                                                                                                             ...
2411    2                 2               boston-la_1088_small_2.mov 70                12                     11323
                                                                                       12                     11324
                                                                                       12                     11325
                                                                                       12                     11326
                                                                                       12                     11327

[250229 rows x 1 columns]

<p><br>

##### Show sample of train/val frame datasets with loaded tensors (to demonstrate functional frame-to-tensor extraction)

In [18]:
df_train_frame_sequences__assoc_index__sample__with_frame_tensors = ib.collect(train_frame_sequences__assoc_index__sample__with_frame_tensors)

In [19]:
df_train_frame_sequences__assoc_index__sample__with_frame_tensors.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX + ['FrameTensor']
df_train_frame_sequences__assoc_index__sample__with_frame_tensors.set_index(fidscs_globals.SCHEMA_PK__TRAIN_OR_VAL_INDEX, inplace=True)
df_train_frame_sequences__assoc_index__sample__with_frame_tensors.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_train_frame_sequences__assoc_index__sample__with_frame_tensors.sort_index(inplace=True)

df_train_frame_sequences__assoc_index__sample__with_frame_tensors

FrameSequence  \
TokenID CameraPerspective ASLConsultantID TargetVideoFilename           UtteranceSequence TokenSequence                  
7       0                 2               muhammed_ali_1052_small_0.mov 2                 0                        264   
        3                 2               muhammed_ali_1052_small_3.mov 23                1                       2889   
10      2                 2               dorm_prank_1053_small_2.mov   1                 3                        101   
188     0                 2               roadtrip1_1050_small_0.mov    25                4                       5854   
319     0                 2               _1583_small_0.mov             51                1                         31   
321     0                 6               ch6-547_181_small_0.mov       4                 3                         68   
324     3                 2               _1493_small_3.mov             174               2                         26   
340     3                 2               roadtrip2_1051_small_3.mov    15                6                       1623   
346     0                 2               siblings_1066_small_0.mov     35                2                       3095   
        2                 2               lapd_story_1083_small_2.mov   90                3                       6783   
461     2                 2               boston-la_1088_small_2.mov    0                 0                         10   
                                          roadtrip2_1051_small_2.mov    16                3                       1678   
                                          whitewater_1049_small_2.mov   44                0                       4020   
474     2                 6               674_479_slave2_small.mov      5                 5                         92   
478     2                 2               football_1073_small_2.mov     17                5                       1945   
481     3                 2               _1637_small_3.mov             126               4                       2631   
497     3                 2               _1628_small_3.mov             96                2                         34   
708     0                 2               lapd_story_1083_small_0.mov   65                5                       4958   
743     0                 2               whitewater_1049_small_0.mov   22                4                       2170   
765     2                 6               ch5-457_157_small_2.mov       32                4                         68   
855     3                 2               muhammed_ali_1052_small_3.mov 13                12                      1660   
1071    2                 2               football_1073_small_2.mov     33                3                       3654   
1208    0                 6               674_479_master_small.mov      5                 6                        113   
        1                 2               _1378_small_1.mov             82                4                         66   
                                          _1596_small_1.mov             64                3                         41   
        2                 2               _1006_small_2.mov             6                 6                         79   
                          3               923_740_slave2_small.mov      8                 5                         69   
                          6               678_484_slave2_small.mov      10                3                         65   
                                          ben_story_441_small_2.mov     26                3                       2046   
        3                 2               _1436_small_3.mov             138               6                         86   
                                          _1578_small_3.mov             46                5                         64   
                                          whitewater_1049_small_3.mov   7      

In [20]:
df_val_frame_sequences_index__sample__with_frame_tensors = ib.collect(val_frame_sequences_index__sample__with_frame_tensors)

In [21]:
df_val_frame_sequences_index__sample__with_frame_tensors.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX + ['FrameTensor']
df_val_frame_sequences_index__sample__with_frame_tensors.set_index(fidscs_globals.SCHEMA_PK__TRAIN_OR_VAL_INDEX, inplace=True)
df_val_frame_sequences_index__sample__with_frame_tensors.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_val_frame_sequences_index__sample__with_frame_tensors.sort_index(inplace=True)

df_val_frame_sequences_index__sample__with_frame_tensors

FrameSequence  \
TokenID CameraPerspective ASLConsultantID TargetVideoFilename          UtteranceSequence TokenSequence                  
9       2                 2               dorm_prank_1053_small_2.mov  25                7                       2611   
248     0                 2               accident_1065_small_0.mov    61                1                       7149   
306     0                 2               whitewater_1049_small_0.mov  2                 4                        295   
320     1                 2               _1340_small_1.mov            45                1                         31   
341     0                 2               boston-la_1088_small_0.mov   42                9                       6906   
355     1                 6               705_510_slave1_small.mov     35                5                         71   
445     1                 2               _1535_small_1.mov            192               5                         85   
461     2                 2               dorm_prank_1053_small_2.mov  15                4                       1472   
                                          lapd_story_1083_small_2.mov  83                1                       6239   
483     1                 2               _1303_small_1.mov            8                 1                         24   
498     0                 2               roadtrip1_1050_small_0.mov   19                0                       4068   
        2                 2               lapd_story_1083_small_2.mov  67                2                       5077   
500     2                 2               lapd_story_1083_small_2.mov  93                2                       7041   
510     1                 6               ben_story_441_small_1.mov    10                3                        793   
                                                                                         3                        798   
530     1                 6               ben_story_443_small_1.mov    23                4                       2075   
583     0                 2               lapd_story_1083_small_0.mov  13                3                       1114   
614     0                 2               accident_1065_small_0.mov    51                7                       6044   
703     2                 2               roadtrip1_1050_small_2.mov   3                 7                        528   
814     1                 6               ben_story_445_small_1.mov    57                4                       1238   
828     3                 2               _1503_small_3.mov            8                 2                         42   
829     2                 2               _1087_small_2.mov            35                0                         15   
835     2                 3               641_slave2_small.mov         2                 0                        135   
1047    2                 2               boston-la_1088_small_2.mov   77                9                      12540   
1208    0                 2               lapd_story_1083_small_0.mov  12                8                       1053   
1211    2                 2               siblings_1066_small_2.mov    24                6                       1846   
1240    1                 6               ben_story_441_small_1.mov    34                5                       2543   
1253    2                 2               lapd_story_1083_small_2.mov  101               6                       7742   
1448    0                 6               660_233_small_0.mov          27                0                         34   
                                          743_550_master_small.mov     24                3                         55   
1449    1                 3               b-965_782_slave1_small.mov   19                0                         15   
1453    0                 3               807_621_master_small.mov     4                 2                     

In [22]:
df_train_frame_sequences_index__sample__with_frame_tensors = ib.collect(train_frame_sequences_index__sample__with_frame_tensors)

In [23]:
df_train_frame_sequences_index__sample__with_frame_tensors.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX + ['FrameTensor']
df_train_frame_sequences_index__sample__with_frame_tensors.set_index(fidscs_globals.SCHEMA_PK__TRAIN_OR_VAL_INDEX, inplace=True)
df_train_frame_sequences_index__sample__with_frame_tensors.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_train_frame_sequences_index__sample__with_frame_tensors.sort_index(inplace=True)

df_train_frame_sequences_index__sample__with_frame_tensors

,,,,,,FrameSequence,FrameTensor
TokenID,CameraPerspective,ASLConsultantID,TargetVideoFilename,UtteranceSequence,TokenSequence,,
7,2,2,lapd_story_1083_small_2.mov,48,0,3713,"[[[[0. 0. 0.12941177], [0. ..."
37,3,2,roadtrip1_1050_small_3.mov,17,11,3761,"[[[[0.09411765 0.09411765 0.2901961 ], [0.1098..."
41,1,2,_1612_small_1.mov,80,4,100,"[[[[0.00784314 0.00392157 0.1254902 ], [0.0274..."
94,1,2,_1415_small_1.mov,117,1,27,"[[[[0.02352941 0.01960784 0.14117648], [0.0392..."
319,2,3,977_794_slave2_small.mov,46,2,34,"[[[[0.03921569 0.01960784 0.13725491], [0.0549..."
406,2,2,lapd_story_1083_small_2.mov,46,3,3622,"[[[[0. 0. 0.12941177], [0. ..."
540,0,2,boston-la_1088_small_0.mov,40,10,6611,"[[[[0. 0. 0.1882353], [0. ..."
683,2,2,boston-la_1088_small_2.mov,37,7,6170,"[[[[0.00392157 0.00392157 0.13725491], [0. ..."
714,0,3,ch6-545_46_small_0.mov,15,3,53,"[[[[0.13725491 0.12941177 0.14117648], [0.1450..."
